# Importamos librerias

In [1]:
import pandas as pd
import numpy as np
import snowflake.connector
import json
import os
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [2]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

    print('Correct Password - connected to SNOWFLAKE')

Connected


# Periodos

In [7]:
periodo = input("Periodo - ejemplo 2023-01:")

# OTROS

In [8]:
query = '''
SELECT
    LPTO.*,
    LPE.EVENTO_DESC,
    FP.PROM_FECHA_INICIO AS INICIO,
    FP.PROM_FECHA_FIN AS FIN,
    CLA.CLASE,
    CLA.CLASS_NAME,
    SUB.SUBCLASE,
    SUB.SUB_NAME,
    LAA.ORIN,
    LAA.ARTC_ARTC_DESC,
    FP.GEOG_LOCL_ID,
    LGL.GEOG_LOCL_COD,
    LGL.GEOG_LOCL_DESC
FROM
    MSTRDB.DWH.FT_PROMOS AS FP
    INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
    INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
    AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
'''

cursor.execute(query.format(periodo_query = periodo))
df = cursor.fetch_pandas_all()
df.head(2)

,TIPO_OFERTA_ID,TIPO_OFERTA_DESC,EVENTO_DESC,INICIO,FIN,CLASE,CLASS_NAME,SUBCLASE,SUB_NAME,ORIN,ARTC_ARTC_DESC,GEOG_LOCL_ID,GEOG_LOCL_COD,GEOG_LOCL_DESC


## Articulos Publicados

### tabla 1 - Articulos Publicados

In [9]:
e_1 = df.groupby(['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'EVENTO_DESC', 'INICIO', 'FIN', 'GEOG_LOCL_COD', 'GEOG_LOCL_DESC','CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['ORIN'].nunique().reset_index()
e_1.rename({'ORIN':'NUMERO_ARTICULOS'}, axis = 1, inplace = True)
e_1.head(2)

,TIPO_OFERTA_ID,TIPO_OFERTA_DESC,EVENTO_DESC,INICIO,FIN,GEOG_LOCL_COD,GEOG_LOCL_DESC,CLASE,CLASS_NAME,SUBCLASE,SUB_NAME,NUMERO_ARTICULOS


## Ventas, GB1 & Numero de Subclases

### Queries

In [10]:
query = '''
WITH PROMOS AS
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        CLA.CLASE,
        CLA.CLASS_NAME,
        SUB.SUBCLASE,
        SUB.SUB_NAME,
        LAA.ORIN,
        LAA.ARTC_ARTC_DESC,
        FP.GEOG_LOCL_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
    )

SELECT
    PROMOS.TIPO_OFERTA_ID,
    PROMOS.TIPO_OFERTA_DESC,
    FV.TIEM_DIA_ID,
    IM.CLASE,
    CLA.CLASS_NAME,
    IM.SUBCLASE,
    SUB.SUB_NAME,
    LAA.ORIN,
    LGL.GEOG_LOCL_COD,
    PROMOS.INICIO,
    PROMOS.FIN,
    FV.VNTA_IMPORTE_SIN_IVA AS VENTA_SIN_IVA,
    FV.VNTA_UNIDADES * FV.VNTA_COSTO_PROM_POND AS COSTO,
    VENTA_SIN_IVA - COSTO AS GB1
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV .ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
    INNER JOIN PROMOS ON PROMOS.ORIN = LAA.ORIN AND PROMOS.GEOG_LOCL_COD = LGL.GEOG_LOCL_COD AND FV.TIEM_DIA_ID BETWEEN PROMOS.INICIO AND PROMOS.FIN
WHERE
    TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') = '{periodo_query}'
'''

cursor.execute(query.format(periodo_query = periodo))
arts_1 = cursor.fetch_pandas_all()
print(arts_1.shape[0])
arts_1.head(2)

0


,TIPO_OFERTA_ID,TIPO_OFERTA_DESC,TIEM_DIA_ID,CLASE,CLASS_NAME,SUBCLASE,SUB_NAME,ORIN,GEOG_LOCL_COD,INICIO,FIN,VENTA_SIN_IVA,COSTO,GB1


In [11]:
query = '''
WITH PROMOS AS
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        CLA.CLASE,
        CLA.CLASS_NAME,
        SUB.SUBCLASE,
        SUB.SUB_NAME,
        LAA.ORIN,
        LAA.ARTC_ARTC_DESC,
        FP.GEOG_LOCL_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
    )

SELECT
    FV.TIEM_DIA_ID,
    IM.CLASE,
    CLA.CLASS_NAME,
    IM.SUBCLASE,
    SUB.SUB_NAME,
    LAA.ORIN,
    LGL.GEOG_LOCL_COD,
    FV.VNTA_IMPORTE_SIN_IVA AS VENTA_SIN_IVA_TOTAL,
    FV.VNTA_UNIDADES * FV.VNTA_COSTO_PROM_POND AS COSTO,
    VENTA_SIN_IVA_TOTAL - COSTO AS GB1_TOTAL
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV .ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
    INNER JOIN PROMOS ON PROMOS.SUBCLASE = IM.SUBCLASE AND PROMOS.GEOG_LOCL_COD = LGL.GEOG_LOCL_COD AND FV.TIEM_DIA_ID BETWEEN PROMOS.INICIO AND PROMOS.FIN
WHERE
    TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') = '{periodo_query}'
'''

cursor.execute(query.format(periodo_query = periodo))
subs_1 = cursor.fetch_pandas_all()
print(subs_1.shape[0])
subs_1.head(2)

0


,TIEM_DIA_ID,CLASE,CLASS_NAME,SUBCLASE,SUB_NAME,ORIN,GEOG_LOCL_COD,VENTA_SIN_IVA_TOTAL,COSTO,GB1_TOTAL


In [12]:
subs_2 = subs_1.groupby(['CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['VENTA_SIN_IVA_TOTAL', 'GB1_TOTAL'].sum().reset_index()

for i in ['VENTA_SIN_IVA_TOTAL', 'GB1_TOTAL']:
    subs_2[i] = subs_2[i].astype(int)

subs_2['REF_2'] = 'Venta Total'

subs_2 = pd.concat([subs_2['REF_2'], subs_2.drop('REF_2', axis=1)], axis=1)
subs_2.head(2)

C:\Users\leonardo.mangold\AppData\Local\Temp\ipykernel_13980\4176392989.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  subs_2 = subs_1.groupby(['CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['VENTA_SIN_IVA_TOTAL', 'GB1_TOTAL'].sum().reset_index()


KeyError: 'VENTA_SIN_IVA_TOTAL'

### tabla 2 - Venta y GB1 por Promo

In [ ]:
arts_2 = arts_1.groupby(['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['VENTA_SIN_IVA', 'GB1'].sum().reset_index()
for i in ['VENTA_SIN_IVA', 'GB1']:
    arts_2[i] = arts_2[i].astype(int)

arts_2['GB1_%'] = round(arts_2['GB1'] / arts_2['VENTA_SIN_IVA'] * 100, 1)
arts_2['REF'] = 'Venta Promo'

arts_2 = pd.concat([arts_2['REF'], arts_2.drop('REF', axis=1)], axis=1)

arts_2.head(2)

### tabla 3 - Numero de Subclases

In [ ]:
arts_2.head(2)
arts_4 = arts_2.groupby(['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC'])['SUBCLASE'].nunique().reset_index()
arts_4.rename({'SUBCLASE':'NUMERO_SUBCLASES'}, axis = 1, inplace = True)
arts_4

In [ ]:
arts_1[['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN', 'GEOG_LOCL_COD']].drop_duplicates()
unicos = arts_1[['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN']].drop_duplicates()

In [ ]:
import pandas as pd

# Assuming 'arts_1' is your original dataframe
# Extracting required columns and dropping duplicates for both dataframes
A = arts_1[['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN', 'GEOG_LOCL_COD']].drop_duplicates()
unicos = arts_1[['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN']].drop_duplicates()

# Merge dataframes on common columns
merged_df = pd.merge(unicos, A, on=['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN'], how='left')

# Group by the specified columns and aggregate 'GEOG_LOCL_COD' values into parentheses
grouped_df = merged_df.groupby(['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN'])['GEOG_LOCL_COD'].agg(
    lambda x: tuple(x) if len(x) > 1 else (x.iloc[0]) if not x.empty else None
).reset_index()

# Rename the aggregated column
grouped_df.rename(columns={'GEOG_LOCL_COD': 'LOCALES'}, inplace=True)

# Merge the grouped dataframe back to the original 'unicos' dataframe
unicos = pd.merge(unicos, grouped_df, on=['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN'], how='left')

unicos['LOCALES'] = list(map(lambda x: str(x).replace('(', '').replace(')', ''), unicos['LOCALES']))
unicos

In [ ]:
arts_5 = pd.DataFrame()

query = '''
SELECT
    DISTINCT
    LGL.GEOG_LOCL_COD,
    IM.SUBCLASE
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FV.GEOG_LOCL_ID
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
WHERE
    FV.TIEM_DIA_ID BETWEEN '{inicio_snow}' AND '{fin_snow}'
    AND LGL.GEOG_LOCL_COD IN ({locales_snow})
'''

for i in range(unicos.shape[0]):
    tipo_oferta_id = unicos.loc[i]['TIPO_OFERTA_ID']
    tipo_oferta_desc = unicos.loc[i]['TIPO_OFERTA_DESC']
    inicio = unicos.loc[i]['INICIO']
    fin = unicos.loc[i]['FIN']
    locales = unicos.loc[i]['LOCALES']

    cursor.execute(query.format(inicio_snow = inicio, fin_snow = fin, locales_snow = locales))
    df_aux = cursor.fetch_pandas_all()
    df_aux['TIPO_OFERTA_ID'] = tipo_oferta_id
    df_aux['TIPO_OFERTA_DESC'] = tipo_oferta_desc
    arts_5 = pd.concat([arts_5, df_aux])

arts_6 = arts_5.groupby(['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC'])['SUBCLASE'].nunique().reset_index()
arts_6.rename({'SUBCLASE':'NUMERO_SUBCLASES_TOTALES'}, axis = 1, inplace = True)

### tabla 3 - Numero de Subclases

In [ ]:
e_6 = arts_4.merge(arts_6, on = ['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC'], how = 'inner')
e_6['RATIO'] = round(e_6['NUMERO_SUBCLASES'] / e_6['NUMERO_SUBCLASES_TOTALES'] * 100, 1)
e_6

### tabla 4 - Venta y GB1 por Subclase

In [ ]:
# Hay subclases que estan presentes en mas de una promo
arts_2[arts_2['SUB_NAME'].duplicated()]
arts_2[arts_2['SUBCLASE'] == 100020003]

In [ ]:
# Para comparar con las ventas de la subclase o clase, tengo que agrupar y perder el tipo de oferta
arts_3 = arts_2.groupby(['REF', 'CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['VENTA_SIN_IVA', 'GB1'].sum().reset_index()
arts_3.head(2)

In [ ]:
e_2 = arts_3.merge(subs_2, on = ['CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'], how = 'inner')
e_2['RATIO_VENTAS'] = round((e_2['VENTA_SIN_IVA'] / e_2['VENTA_SIN_IVA_TOTAL']) * 100, 1)
e_2['RATIO_GB1'] = round((e_2['GB1'] / e_2['GB1_TOTAL']) * 100, 1)
e_2.head(2)

### tabla 5 - Venta y GB1 Promo Agrupado

In [ ]:
e_3 = pd.DataFrame(e_2.sum(axis=0)).T
e_3['REF'] = 'Venta Promo'
e_3['VENTA_SIN_IVA_MILLONES'] = int(round(e_3['VENTA_SIN_IVA'].astype(int) / 1000000, 0))
e_3['GB1_MILLONES'] = int(round(e_3['GB1'].astype(int) / 1000000, 0))
e_3.drop(['REF_2'], axis = 1, inplace = True)
e_3['REF_2'] = 'Venta Total'
e_3['VENTA_SIN_IVA_TOTAL_MILLONES'] = int(round(e_3['VENTA_SIN_IVA_TOTAL'].astype(int) / 1000000, 0))
e_3['GB1_TOTAL_MILLONES'] = int(round(e_3['GB1_TOTAL'].astype(int) / 1000000, 0))
e_3.drop(['CLASE', 'SUBCLASE', 'GB1', 'VENTA_SIN_IVA', 'VENTA_SIN_IVA_TOTAL', 'GB1_TOTAL', 'RATIO_VENTAS', 'RATIO_GB1', 'CLASS_NAME', 'SUB_NAME'], axis = 1, inplace = True)
e_3['RATIO_VENTAS'] = round((e_3['VENTA_SIN_IVA_MILLONES'] / e_3['VENTA_SIN_IVA_TOTAL_MILLONES']) * 100, 1)
e_3['RATIO_GB1'] = round((e_3['GB1_MILLONES'] / e_3['GB1_TOTAL_MILLONES']) * 100, 1)
e_3

## Cantidad de Articulos

### Queries

In [ ]:
query = '''
WITH PROMOS AS
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        CLA.CLASE,
        CLA.CLASS_NAME,
        SUB.SUBCLASE,
        SUB.SUB_NAME,
        LAA.ORIN,
        LAA.ARTC_ARTC_DESC,
        FP.GEOG_LOCL_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
    )

SELECT
    PROMOS.TIPO_OFERTA_ID,
    PROMOS.TIPO_OFERTA_DESC,
    FV.TIEM_DIA_ID,
    IM.CLASE,
    CLA.CLASS_NAME,
    IM.SUBCLASE,
    SUB.SUB_NAME,
    LAA.ORIN,
    LGL.GEOG_LOCL_COD,
    FV.VNTA_UNIDADES AS UNIDADES_PROMO
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV .ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
    INNER JOIN PROMOS ON PROMOS.ORIN = LAA.ORIN AND PROMOS.GEOG_LOCL_COD = LGL.GEOG_LOCL_COD AND FV.TIEM_DIA_ID BETWEEN PROMOS.INICIO AND PROMOS.FIN
WHERE
    TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') = '{periodo_query}'
'''

cursor.execute(query.format(periodo_query = periodo))
cant_1 = cursor.fetch_pandas_all()
cant_1['REF'] = 'Venta Promo'
print(cant_1.shape[0])
cant_1.head(2)

In [ ]:
# Para comparar con las ventas de la subclase o clase, tengo que agrupar y perder el tipo de oferta
cant_2 = cant_1.groupby(['REF', 'CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['UNIDADES_PROMO'].sum().reset_index()
cant_2.head(2)

In [ ]:
query = '''
WITH PROMOS AS
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        CLA.CLASE,
        CLA.CLASS_NAME,
        SUB.SUBCLASE,
        SUB.SUB_NAME,
        LAA.ORIN,
        LAA.ARTC_ARTC_DESC,
        FP.GEOG_LOCL_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
    )

SELECT
    FV.TIEM_DIA_ID,
    IM.CLASE,
    CLA.CLASS_NAME,
    IM.SUBCLASE,
    SUB.SUB_NAME,
    LAA.ORIN,
    LGL.GEOG_LOCL_COD,
    FV.VNTA_UNIDADES AS UNIDADES_TOTALES
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV .ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
    INNER JOIN PROMOS ON PROMOS.SUBCLASE = IM.SUBCLASE AND PROMOS.GEOG_LOCL_COD = LGL.GEOG_LOCL_COD AND FV.TIEM_DIA_ID BETWEEN PROMOS.INICIO AND PROMOS.FIN
WHERE
    TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') = '{periodo_query}'
'''

cursor.execute(query.format(periodo_query = periodo))
sub_cant_1 = cursor.fetch_pandas_all()
sub_cant_1['REF_2'] = 'Venta Total'
print(sub_cant_1.shape[0])
sub_cant_1.head(2)

In [ ]:
sub_cant_2 = sub_cant_1.groupby(['REF_2','CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['UNIDADES_TOTALES'].sum().reset_index()
sub_cant_2.head(2)

### tabla 6 - Num Articulos por Subclase

In [ ]:
e_4 = cant_2.merge(sub_cant_2, on = ['CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'], how = 'inner')

e_4['RATIO_UNIDADES'] = round(e_4['UNIDADES_PROMO'] / e_4['UNIDADES_TOTALES'] * 100, 1)
for i in ['UNIDADES_PROMO', 'UNIDADES_TOTALES']:
    e_4[i] = e_4[i].astype(int)

e_4.head(2)

## Aceleracion

In [ ]:
query = '''
WITH DATOS AS
    (
    SELECT
        DISTINCT
        FP.EVENTO_ID,
        LPE.EVENTO_DESC,
        LPTO.TIPO_OFERTA_DESC,
        FP.PROM_FECHA_INICIO,
        FP.PROM_FECHA_FIN,
        DATEDIFF('DAYS', FP.PROM_FECHA_INICIO, FP.PROM_FECHA_FIN) AS DURACION_DIAS,
        FP.ARTC_ARTC_ID,
        FP.GEOG_LOCL_ID,
        FP.ARTC_ARTC_ID || FP.GEOG_LOCL_ID AS CLAVE
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        LEFT JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        LEFT JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
    WHERE
        LGL.GEOG_UNNG_ID = 2
        AND LGL.GEOG_LOCL_COD <> 198
    ),

BASAL AS
    (
    SELECT
        SPLIT_PART(TIEM_DIA_ID, ' ', 1) AS TIEM_DIA_ID,
        GEOG_LOCL_ID,
        ARTC_ARTC_ID,
        SUM(VENTA_BASAL) AS VENTA_BASAL
    FROM
        (
            SELECT
                FVB.TIEM_DIA_ID,
                FVB.GEOG_LOCL_ID,
                FVB.ARTC_ARTC_ID,
                FVB.VENTA_BASAL
            FROM
                BIZMETRIKS.DWH.FT_VENTA_BASAL AS FVB
                INNER JOIN DATOS AS D ON FVB.GEOG_LOCL_ID = D.GEOG_LOCL_ID AND FVB.ARTC_ARTC_ID = D.ARTC_ARTC_ID AND FVB.TIEM_DIA_ID BETWEEN D.PROM_FECHA_INICIO AND D.PROM_FECHA_FIN
            WHERE
                FVB.TIEM_DIA_ID > DATEADD(YEAR, -1, CURRENT_DATE())
            )
        GROUP BY
            ALL
    ),

PROMOS AS
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        CLA.CLASE,
        CLA.CLASS_NAME,
        SUB.SUBCLASE,
        SUB.SUB_NAME,
        LAA.ORIN,
        LAA.ARTC_ARTC_DESC,
        FP.GEOG_LOCL_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
    ),

PROMOS_2 AS
    (
    SELECT
        FV.TIEM_DIA_ID,
        IM.CLASE,
        CLA.CLASS_NAME,
        IM.SUBCLASE,
        SUB.SUB_NAME,
        LAA.ARTC_ARTC_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_ID,
        SUM(FV.VNTA_UNIDADES) AS VNTA_UNIDADES
    FROM
        MSTRDB.DWH.FT_VENTAS AS FV
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV .ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN PROMOS ON PROMOS.ORIN = LAA.ORIN AND PROMOS.GEOG_LOCL_COD = LGL.GEOG_LOCL_COD AND FV.TIEM_DIA_ID BETWEEN PROMOS.INICIO AND PROMOS.FIN
    WHERE
        TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') = '{periodo_query}'
    GROUP BY
        ALL
    )

SELECT
    PROMOS_2.*,
    BASAL.VENTA_BASAL
FROM
    PROMOS_2
    LEFT JOIN BASAL ON BASAL.TIEM_DIA_ID = PROMOS_2.TIEM_DIA_ID AND BASAL.GEOG_LOCL_ID = PROMOS_2.GEOG_LOCL_ID AND BASAL.ARTC_ARTC_ID = PROMOS_2.ARTC_ARTC_ID
'''

cursor.execute(query.format(periodo_query = periodo))
ace_perdida = cursor.fetch_pandas_all()
ace_perdida.head(2)

In [ ]:
print(f"Filas con Basal missing: {ace_perdida[ace_perdida['VENTA_BASAL'].isna()].shape[0]}")
print(f"missing para {periodo}: {int(round(ace_perdida[ace_perdida['VENTA_BASAL'].isna()].shape[0] / ace_perdida.shape[0] * 100, 0))}%")

In [ ]:
query = '''
WITH DATOS AS
    (
    SELECT
        DISTINCT
        FP.EVENTO_ID,
        LPE.EVENTO_DESC,
        LPTO.TIPO_OFERTA_DESC,
        FP.PROM_FECHA_INICIO,
        FP.PROM_FECHA_FIN,
        DATEDIFF('DAYS', FP.PROM_FECHA_INICIO, FP.PROM_FECHA_FIN) AS DURACION_DIAS,
        FP.ARTC_ARTC_ID,
        FP.GEOG_LOCL_ID,
        FP.ARTC_ARTC_ID || FP.GEOG_LOCL_ID AS CLAVE
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        LEFT JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        LEFT JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
    WHERE
        LGL.GEOG_UNNG_ID = 2
        AND LGL.GEOG_LOCL_COD <> 198
    ),

BASAL AS
    (
    SELECT
        SPLIT_PART(TIEM_DIA_ID, ' ', 1) AS TIEM_DIA_ID,
        GEOG_LOCL_ID,
        ARTC_ARTC_ID,
        SUM(VENTA_BASAL) AS VENTA_BASAL
    FROM
        (
            SELECT
                FVB.TIEM_DIA_ID,
                FVB.GEOG_LOCL_ID,
                FVB.ARTC_ARTC_ID,
                FVB.VENTA_BASAL
            FROM
                BIZMETRIKS.DWH.FT_VENTA_BASAL AS FVB
                INNER JOIN DATOS AS D ON FVB.GEOG_LOCL_ID = D.GEOG_LOCL_ID AND FVB.ARTC_ARTC_ID = D.ARTC_ARTC_ID AND FVB.TIEM_DIA_ID BETWEEN D.PROM_FECHA_INICIO AND D.PROM_FECHA_FIN
            WHERE
                FVB.TIEM_DIA_ID > DATEADD(YEAR, -1, CURRENT_DATE())
            )
        GROUP BY
            ALL
    ),

PROMOS AS
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        CLA.CLASE,
        CLA.CLASS_NAME,
        SUB.SUBCLASE,
        SUB.SUB_NAME,
        LAA.ORIN,
        LAA.ARTC_ARTC_COD,
        LAA.ARTC_ARTC_DESC,
        FP.GEOG_LOCL_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
    ),

PROMOS_2 AS
    (
    SELECT
        FV.TIEM_DIA_ID,
        IM.CLASE,
        CLA.CLASS_NAME,
        IM.SUBCLASE,
        SUB.SUB_NAME,
        LAA.ARTC_ARTC_ID,
        LAA.ARTC_ARTC_COD,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_ID,
        SUM(FV.VNTA_UNIDADES) AS VNTA_UNIDADES
    FROM
        MSTRDB.DWH.FT_VENTAS AS FV
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV .ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN PROMOS ON PROMOS.ORIN = LAA.ORIN AND PROMOS.GEOG_LOCL_COD = LGL.GEOG_LOCL_COD AND FV.TIEM_DIA_ID BETWEEN PROMOS.INICIO AND PROMOS.FIN
    WHERE
        TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') = '{periodo_query}'
    GROUP BY
        ALL
    )

SELECT
    PROMOS_2.*,
    BASAL.VENTA_BASAL
FROM
    PROMOS_2
    INNER JOIN BASAL ON BASAL.TIEM_DIA_ID = PROMOS_2.TIEM_DIA_ID AND BASAL.GEOG_LOCL_ID = PROMOS_2.GEOG_LOCL_ID AND BASAL.ARTC_ARTC_ID = PROMOS_2.ARTC_ARTC_ID
'''

cursor.execute(query.format(periodo_query = periodo))
ace = cursor.fetch_pandas_all()
ace.head(2)

### tabla 7 - Aceleracion

In [ ]:
ace['DIFF'] = ace['VNTA_UNIDADES'] - ace['VENTA_BASAL']

ace['UNIDADES_DIV_BASAL'] = round(ace['VNTA_UNIDADES'] / ace['VENTA_BASAL'], 1)
for i in ['DIFF', 'VNTA_UNIDADES', 'VENTA_BASAL']:
    ace[i] = ace[i].astype(int)
ace.head(2)

### tabla 8 - Aceleracion por Subclase

In [ ]:
ace_2 = ace.groupby(['CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['VNTA_UNIDADES', 'VENTA_BASAL'].sum().reset_index()

ace_2['DIFF'] = ace_2['VNTA_UNIDADES'] - ace_2['VENTA_BASAL']
ace_2['UNIDADES_DIV_BASAL'] = round(ace_2['VNTA_UNIDADES'] / ace_2['VENTA_BASAL'], 1)
ace_2.head(2)

### tabla 9 - Aceleracion Resumen

In [ ]:
ace_3 = pd.DataFrame(ace[['VNTA_UNIDADES', 'VENTA_BASAL']].sum(axis=0)).T
ace_3['DIFF'] = ace_3['VNTA_UNIDADES'] - ace_3['VENTA_BASAL']
ace_3['UNIDADES_DIV_BASAL'] = round(ace_3['VNTA_UNIDADES'] / ace_3['VENTA_BASAL'], 1)
ace_3

## Tasa de Cierre

### Promos

In [ ]:
query = '''
WITH PROMOS AS
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        CLA.CLASE,
        CLA.CLASS_NAME,
        SUB.SUBCLASE,
        SUB.SUB_NAME,
        LAA.ORIN,
        LAA.ARTC_ARTC_DESC,
        FP.GEOG_LOCL_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
    )

SELECT
    FV.TIEM_DIA_ID,
    IM.CLASE,
    CLA.CLASS_NAME,
    IM.SUBCLASE,
    SUB.SUB_NAME,
    LAA.ORIN,
    LGL.GEOG_LOCL_COD,
    COUNT(DISTINCT FV.TICKET) AS TICKETS
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV .ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
    INNER JOIN PROMOS ON PROMOS.ORIN = LAA.ORIN AND PROMOS.GEOG_LOCL_COD = LGL.GEOG_LOCL_COD AND FV.TIEM_DIA_ID BETWEEN PROMOS.INICIO AND PROMOS.FIN
WHERE
    TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') = '{periodo_query}'
GROUP BY
    ALL
'''

cursor.execute(query.format(periodo_query = periodo))
tick_1 = cursor.fetch_pandas_all()
print(tick_1.shape[0])
tick_1.head(2)

In [ ]:
tick_2 = tick_1.groupby(['CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['TICKETS'].sum().reset_index()
tick_2.rename({'TICKETS':'TICKETS_PROMO'}, axis = 1, inplace = True)
tick_2.head(2)

### Para toda la subclase en dicho local

In [ ]:
query = '''
WITH PROMOS AS
    (
    SELECT
        LPTO.*,
        LPE.EVENTO_DESC,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        CLA.CLASE,
        CLA.CLASS_NAME,
        SUB.SUBCLASE,
        SUB.SUB_NAME,
        LAA.ORIN,
        LAA.ARTC_ARTC_DESC,
        FP.GEOG_LOCL_ID,
        LGL.GEOG_LOCL_COD,
        LGL.GEOG_LOCL_DESC
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '{periodo_query}' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '{periodo_query}')
    )

SELECT
    FV.TIEM_DIA_ID,
    IM.CLASE,
    CLA.CLASS_NAME,
    IM.SUBCLASE,
    SUB.SUB_NAME,
    LAA.ORIN,
    LGL.GEOG_LOCL_COD,
    COUNT(DISTINCT FV.TICKET) AS TICKETS
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV .ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
    INNER JOIN PROMOS ON PROMOS.SUBCLASE = IM.SUBCLASE AND PROMOS.GEOG_LOCL_COD = LGL.GEOG_LOCL_COD AND FV.TIEM_DIA_ID BETWEEN PROMOS.INICIO AND PROMOS.FIN
WHERE
    TO_VARCHAR(FV.TIEM_DIA_ID, 'YYYY-MM') = '{periodo_query}'
GROUP BY
    ALL
'''

cursor.execute(query.format(periodo_query = periodo))
tick_total_1 = cursor.fetch_pandas_all()
print(tick_total_1.shape[0])
tick_total_1.head(2)

In [ ]:
tick_total_2 = tick_total_1.groupby(['CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'])['TICKETS'].sum().reset_index()
tick_total_2.rename({'TICKETS':'TICKETS_TOTALES'}, axis = 1, inplace = True)
tick_total_2.head(2)

### tabla 10 - Tasa de Cierre

In [ ]:
e_5 = tick_total_2.merge(tick_2, on = ['CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME'], how = 'left')
for i in ['TICKETS_TOTALES', 'TICKETS_PROMO']:
    e_5[i].fillna(0, inplace = True)
    e_5[i] = e_5[i].astype(int)
e_5['RATIO_TICKETS'] = round((e_5['TICKETS_PROMO'] / e_5['TICKETS_TOTALES']) * 100, 1)
e_5 =e_5[['CLASE', 'CLASS_NAME', 'SUBCLASE', 'SUB_NAME', 'TICKETS_PROMO', 'TICKETS_TOTALES', 'RATIO_TICKETS']]
e_5.head(2)

# MAURI

## Promos Distintas

### tabla 11 - Numero de Eventos

In [ ]:
query = '''
    SELECT
        LPTO.*,
        COUNT(DISTINCT LPE.EVENTO_ID) AS NUMERO_EVENTOS
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
        AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM') = '2023-01' OR TO_VARCHAR(FP.PROM_FECHA_FIN, 'YYYY-MM') = '2023-01')
    GROUP BY
        ALL
'''

cursor.execute(query)
eventos = cursor.fetch_pandas_all()
eventos

## Precios Oferta

### tabla 12 - Precios Oferta

In [3]:
url_fechas = 'https://docs.google.com/spreadsheets/d/1JnayuiljvaOebik4gKqNgxSLD6RO2AS8euG7apQykW8/export?format=csv'
fechas = pd.read_csv(url_fechas)
fechas

,EVENTO_ID,DESDE,HASTA,OFERTA
0,NaN,2024-01-04,2024-01-10,OFERTATA ENERO 01
1,NaN,2024-01-11,2024-01-17,OFERTATA ENERO 02
2,NaN,2024-02-15,2024-02-21,OFERTATA FEBRERO 07
3,NaN,2024-02-22,2024-02-28,OFERTATA FEBRERO 08


In [6]:
query = '''
    SELECT
        DISTINCT
        LPTO.*,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        --LAA.ARTC_ARTC_ID,
        --LAA.ARTC_ARTC_COD,
        LAA.ORIN,
        --LAA.ARTC_ARTC_DESC,
        FP.PROM_PVP_OFERTA
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FP.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    WHERE
        LPTO.TIPO_OFERTA_ID in (10,11,12)
        AND FP.PROM_FECHA_INICIO >= DATEADD(DAY, -30, '{desde_snow}')
        AND FP.PROM_FECHA_FIN < '{desde_snow}'
'''

df_precios_oferta = pd.DataFrame()
for i in range(len(fechas)):
    desde = fechas.iloc[i]['DESDE']
    hasta = fechas.iloc[i]['HASTA']

    cursor.execute(query.format(desde_snow = desde, hasta_snow = hasta))
    df_aux = cursor.fetch_pandas_all()
    df_aux['PRECIO DESDE'] = desde
    df_aux['PRECIO HASTA'] = hasta
    df_precios_oferta = pd.concat([df_precios_oferta, df_aux])
df_precios_oferta = df_precios_oferta[['TIPO_OFERTA_ID', 'TIPO_OFERTA_DESC', 'INICIO', 'FIN', 'PRECIO DESDE', 'PRECIO HASTA', 'ORIN', 'PROM_PVP_OFERTA']]
df_precios_oferta.head(2)

,TIPO_OFERTA_ID,TIPO_OFERTA_DESC,INICIO,FIN,PRECIO DESDE,PRECIO HASTA,ORIN,PROM_PVP_OFERTA
0,12,Otros Mailings,2023-12-26,2024-01-03,2024-01-04,2024-01-10,1000052069,40.0
1,12,Otros Mailings,2023-12-26,2024-01-03,2024-01-04,2024-01-10,1000149602,135.0


In [7]:
df_precios_oferta['FIN'] = pd.to_datetime(df_precios_oferta['FIN'])

# Create a new 'Rank' column based on the maximum date for each element of 'X'
df_precios_oferta['R'] = df_precios_oferta.groupby('ARTC_ARTC_ID')['FIN'].rank(ascending=False, method='max').astype(int)
print(df_precios_oferta['ARTC_ARTC_ID'].nunique())

df_precios_oferta = df_precios_oferta[df_precios_oferta['R'] == 1]
print(df_precios_oferta['ARTC_ARTC_ID'].nunique())
df_precios_oferta.drop_duplicates(subset = 'ARTC_ARTC_ID', keep = 'first', inplace = True)
df_precios_oferta.drop(['R'], axis = 1, inplace = True)
print(df_precios_oferta.shape[0])

KeyError: 'ARTC_ARTC_ID'

In [ ]:
df_precios_oferta[df_precios_oferta.duplicated(subset = 'ARTC_ARTC_ID', keep = False)]

## BASE_OPT

### tabla 13 - OPT

In [ ]:
query = '''
    SELECT
        LAA.ARTC_ARTC_ID,
        LAA.ARTC_ARTC_COD,
        LAA.ARTC_ARTC_DESC,
        --LAA.ORIN,
        ROUND(AVG(OPT.PRECIO_COMPETENCIA), 1) AS AVG_PRECIO_COMPETENCIA
    FROM
        SPIKE.SPIKE.BASE_OPT AS OPT
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON OPT.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON OPT.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
    GROUP BY
        ALL
'''

cursor.execute(query)
opt = cursor.fetch_pandas_all()
opt.head(2)

## Locales Activos Ayer

### tabla 14 - Locales Activos Ayer

In [43]:
query = '''
SELECT
    --LAA.ARTC_ARTC_DESC AS ARTICULO,
    LAA.ORIN,
    --LAA.ARTC_ARTC_ID,
    COUNT(DISTINCT FS.GEOG_LOCL_ID) AS LOCALES_ACTIVOS_AYER
FROM
    MSTRDB.DWH.FT_STOCK AS FS
    LEFT JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FS.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
WHERE
    ARTC_ESTA_ID = 4
    AND GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_LOCL_COD NOT IN (198, 100))
    AND FS.TIEM_DIA_ID = CURRENT_DATE() - 1
GROUP BY
    ALL
'''

cursor.execute(query)
e_7 = cursor.fetch_pandas_all()
e_7.head(2)

,ORIN,LOCALES_ACTIVOS_AYER
0,1000014196,1
1,1000050646,83


In [44]:
e_7['LOCALES_ACTIVOS_AYER'].unique()

array([ 1, 83, 89, 91, 70, 75, 93, 92, 71, 58, 94, 90, 69, 22, 67, 33, 50,
       51, 60, 61, 72,  5, 49, 85, 86, 62, 38, 65, 74, 48, 68, 35, 87, 52,
       81, 30, 79, 44, 47, 77, 29, 37, 40, 63, 73, 43, 80, 57, 88, 27, 26,
       39, 64, 66, 15, 55, 42, 82, 34, 53, 78, 59, 28, 10, 31, 21, 36, 54,
       45, 41, 84, 20, 76, 46, 19, 32,  2,  8, 25, 24, 23, 18, 56,  9, 12,
       95,  4,  3, 17, 14,  7, 11, 13, 16,  6], dtype=int64)

## Days on Hand

### tabla 15 - days on hand ARTICULO

In [8]:
query = '''
WITH STOCK AS
    (
    SELECT
        IM.SUBCLASE,
        LAA.ARTC_ARTC_DESC,
        LAA.ORIN,
        LAA.ARTC_ARTC_ID,
        SUM(FS.STCK_UNIDADES) AS UNIDADES
    FROM
        MSTRDB.DWH.FT_STOCK AS FS
        LEFT JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FS.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON LAA.ORIN = IM.ITEM
    WHERE
        FS.TIEM_DIA_ID = CURRENT_DATE() - 1
    GROUP BY
        ALL
    ),

VENTAS AS
    (
    SELECT
        FV.ARTC_ARTC_ID,
        (SUM(FV.VNTA_UNIDADES) / 30) AS UNIDADES_VENDIDAS
    FROM
        MSTRDB.DWH.FT_VENTAS AS FV
    WHERE
        FV.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE()-1)
    GROUP BY
        ALL
    )

SELECT
    STOCK.*,
    COALESCE(VENTAS.UNIDADES_VENDIDAS, 0) AS UNIDADES_VENDIDAS
FROM
    STOCK
    LEFT JOIN VENTAS ON VENTAS.ARTC_ARTC_ID = STOCK.ARTC_ARTC_ID
'''

cursor.execute(query)
e_8 = cursor.fetch_pandas_all()
e_8 = e_8[e_8['UNIDADES'] != 0]
e_8['DAYS ON HAND'] = e_8['UNIDADES'] / e_8['UNIDADES_VENDIDAS']
e_8['DAYS ON HAND'][(e_8['UNIDADES'] == 0) & (e_8['UNIDADES_VENDIDAS'] == 0)] = 0
e_8['DAYS ON HAND'] = round(e_8['DAYS ON HAND'], 2)
e_8['UNIDADES'] = round(e_8['UNIDADES'], 2)
e_8['UNIDADES_VENDIDAS'] = round(e_8['UNIDADES_VENDIDAS'], 2)
e_8 = e_8[['SUBCLASE', 'ORIN', 'DAYS ON HAND']]
e_8.head(2)

,SUBCLASE,ARTC_ARTC_DESC,ORIN,ARTC_ARTC_ID,UNIDADES,UNIDADES_VENDIDAS
0,1500050002,CAVIAR NEGRO STUK DEUSTECHER 50.00 U 1,1000054044,69378,0.0,0.0
1,1600040003,COPOS DE MAIZ NATURALES MULTIAHORRO 200.00 G 1,1000054303,69450,0.0,0.0


### tabla 16 - days on hand SUBCLASE

In [17]:
query = '''
WITH STOCK AS
    (
    SELECT
        SUB.SUBCLASE,
        SUB.SUB_NAME,
        SUM(FS.STCK_UNIDADES) AS UNIDADES
    FROM
        MSTRDB.DWH.FT_STOCK AS FS
        LEFT JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FS.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        FS.TIEM_DIA_ID = CURRENT_DATE() - 1
    GROUP BY
        ALL
    ),

VENTAS AS
    (
    SELECT
        SUB.SUBCLASE,
        (SUM(FV.VNTA_UNIDADES) / 30) AS UNIDADES_VENDIDAS
    FROM
        MSTRDB.DWH.FT_VENTAS AS FV
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
        INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    WHERE
        FV.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE()-1)
    GROUP BY
        ALL
    )

SELECT
    STOCK.*,
    COALESCE(VENTAS.UNIDADES_VENDIDAS, 0) AS UNIDADES_VENDIDAS
FROM
    STOCK
    LEFT JOIN VENTAS ON VENTAS.SUBCLASE = STOCK.SUBCLASE
'''

cursor.execute(query)
e_9 = cursor.fetch_pandas_all()
e_9 = e_9[e_9['UNIDADES'] != 0]
e_9['DAYS ON HAND'] = e_9['UNIDADES'] / e_9['UNIDADES_VENDIDAS']
e_9['DAYS ON HAND'][(e_9['UNIDADES'] == 0) & (e_9['UNIDADES_VENDIDAS'] == 0)] = 0
e_9['DAYS ON HAND'] = round(e_9['DAYS ON HAND'], 2)
e_9['UNIDADES'] = round(e_9['UNIDADES'], 2)
e_9['UNIDADES_VENDIDAS'] = round(e_9['UNIDADES_VENDIDAS'], 2)
e_9 = e_9[['SUBCLASE', 'ORIN', 'DAYS ON HAND']]
e_9.head(2)

,SUBCLASE,SUB_NAME,UNIDADES,UNIDADES_VENDIDAS,DAYS ON HAND
0,8900010008,PANOS ABSORVENTES,34238.0,533.47,64.18
1,6900010001,ARMA DE DARDOS O MUNICIONES,1278.0,10.60,120.57


In [22]:
e_9['DAYS ON HAND'][e_9['DAYS ON HAND'] == np.inf] = 99999999
e_9[e_9['DAYS ON HAND'] == np.inf]

,SUBCLASE,ARTC_ARTC_DESC,ORIN,ARTC_ARTC_ID,UNIDADES,UNIDADES_VENDIDAS,DAYS ON HAND


## Precios Stock Mediano ayer

### tabla 17 - Precios Stock Mediano ayer

In [ ]:
query = '''
    SELECT
        DISTINCT
        LPTO.*,
        FP.PROM_FECHA_INICIO AS INICIO,
        FP.PROM_FECHA_FIN AS FIN,
        --LAA.ARTC_ARTC_ID,
        --LAA.ARTC_ARTC_COD,
        --LAA.ARTC_ARTC_DESC,
        LAA.ORIN,
        MEDIAN(FS.STCK_PRECIO_VENTA_DIA_CIVA) AS PRECIO_STOCK_MEDIANO
    FROM
        MSTRDB.DWH.FT_PROMOS AS FP
        INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
        INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FP.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.FT_STOCK AS FS ON FS.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID AND FS.TIEM_DIA_ID = CURRENT_DATE - 1
    WHERE
        LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
    GROUP BY
        ALL
'''

cursor.execute(query)
df_precios_stock_mediano = cursor.fetch_pandas_all()
df_precios_stock_mediano.head(2)

In [ ]:
# Mauri

with pd.ExcelWriter(
        f"C:\\Users\\leonardo.mangold\\PycharmProjects\\analisis\\kpis_promos_mauri\\Entregas\\Promos {datetime.today().strftime('%Y-%m-%d')}.xlsx",
        engine='xlsxwriter') as writer:



    precios.to_excel(writer, sheet_name=f"Precios Oferta {datetime.today().strftime('%Y-%m-%d')}", index=False)  # tabla 12
    df_precios_stock_mediano.to_excel(writer, sheet_name=f"Precios Stock Mediano Ayer", index=False)  # tabla 17
    opt.to_excel(writer, sheet_name=f"OPT - Media Precios Compet", index=False)  # tabla 13
    e_7.to_excel(writer, sheet_name=f"Numero de Locales Activos Ayer", index=False)  # tabla 14
    e_8.to_excel(writer, sheet_name=f"Days on Hand - Articulos", index=False)  # tabla 15
    e_9.to_excel(writer, sheet_name=f"Days on Hand - Subclases", index=False)  # tabla 16


In [ ]:
sys.exit()

In [ ]:
with pd.ExcelWriter(f"C:\\Users\\leonardo.mangold\\PycharmProjects\\analisis\\kpis_promos_mauri\\Entregas\\Promos {datetime.today().strftime('%Y-%m-%d')}.xlsx", engine='xlsxwriter') as writer:
    e_1.to_excel(writer, sheet_name='Articulos Publicados', index=False) # tabla 1
    arts_2.to_excel(writer, sheet_name='Venta y GB1 por Promo', index=False) # tabla 2
    e_6.to_excel(writer, sheet_name='Numero de Subclases', index=False) # tabla 3
    e_2.to_excel(writer, sheet_name='Venta y GB1 por Subclase', index=False) # tabla 4
    e_3.to_excel(writer, sheet_name='Venta y GB1 Promo Agrupado', index=False) # tabla 5
    e_4.to_excel(writer, sheet_name='Num Articulos por Subclase', index=False) # tabla 6
    ace.to_excel(writer, sheet_name='Aceleracion', index=False) # tabla 7
    ace_2.to_excel(writer, sheet_name='Aceleracion por Subclase', index=False) # tabla 8
    ace_3.to_excel(writer, sheet_name='Aceleracion Resumen', index=False) # tabla 9
    e_5.to_excel(writer, sheet_name='Tickets', index=False) # tabla 10
    eventos.to_excel(writer, sheet_name='Numero de Eventos', index=False) # tabla 11
    df_precios_oferta.to_excel(writer, sheet_name=f"Precios {datetime.today().strftime('%Y-%m-%d')}", index=False) # tabla 12
    opt.to_excel(writer, sheet_name=f"OPT - Media Precios Compet", index=False) # tabla 13
    e_7.to_excel(writer, sheet_name=f"Numero de Locales Activos Ayer", index=False) # tabla 14
    e_8.to_excel(writer, sheet_name=f"Days on Hand - Articulos", index=False) # tabla 15
    e_9.to_excel(writer, sheet_name=f"Days on Hand - Subclases", index=False) # tabla 16


In [14]:
datetime.today().strftime('%Y-%m-%d')

'2023-12-13'

# Fin